In [48]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
from PIL import Image

In [103]:
# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, transform=None):
        "Initialization"
        self.transform = transform
        #self.frames = frames
        self.folders = data_path
        self.frames = 10 #For our case since we are computing 10 frames always

    def __len__(self):
        "Denotes the total number of samples"
        return len(os.listdir(self.folders))

    def read_images(self, data_path, use_transform):
        X = []
        for i in os.listdir(data_path):
            #print("file name is ",i)
            image = Image.open(os.path.join(data_path,i))
            
            #print(image.shape)
            if use_transform is not None:
                image = use_transform(image)
                #print(image.size)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)

        return X

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
              
        # Load data
        X = self.read_images(data_path, self.transform)                     # (input) spatial images
        
        y = np.ones(self.frames)
        if 'orig' in data_path:
            y = np.zeros(self.frames)
        # print(X.shape)
        return X, torch.from_numpy(y).type(torch.LongTensor)

In [104]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    #transforms.CenterCrop(256),
    #transforms.ToTensor()
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225] )
    ])

In [105]:
train_path = '/home/chinmay/datatset/train/'
train_data = Dataset_CRNN(train_path, transform=TRANSFORM_IMG)
# for step, (x, y) in enumerate(data):
#     print(x.shape)
val_path = '/home/chinmay/datatset/val/'
val_data = Dataset_CRNN(val_path, transform=TRANSFORM_IMG)

In [106]:
def new_output_shape(num_of_maxpool_2,shape):
    return int(shape/(2**num_of_maxpool_2))

In [107]:
print(new_output_shape(4,256))

16


In [108]:
class Meso4(nn.Module):
    def __init__(self,in_channel=3, img_shape = 256, number_of_classes=2):
        super(Meso4,self).__init__()
        self.conv1 = nn.Conv2d(in_channel,8, kernel_size=(3,3), stride = 1, padding= 1)
        self.batch_norm_1 = nn.BatchNorm2d(8)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(8,8, kernel_size=(5,5),stride=1, padding=2)
        self.batch_norm_2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,16, kernel_size=(5,5),stride=1, padding=2)
        self.batch_norm_3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16,16, kernel_size=(5,5),stride=1, padding=2)
        self.batch_norm_4 = nn.BatchNorm2d(16)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.max_pool_4 = nn.MaxPool2d(kernel_size=(4, 4), stride=4)
        out_shape = new_output_shape(num_of_maxpool_2=4, shape = img_shape)
        self.fc_conv = nn.Conv2d(16,number_of_classes,kernel_size=(out_shape,out_shape))
        
    def forward(self,x_3d):
        cnn_embed_seq = []
        x_3d = x_3d.permute(0,1,4,2,3)# Required to match shapes
        x_3d = x_3d.type(torch.cuda.FloatTensor) #Converting to Float Tensor from Byte Tensor
        for t in range(x_3d.size(1)):
            x = self.conv1(x_3d[:, t, :, :, :])
            x = self.batch_norm_1(x)
            x = self.relu(x)
            x = self.max_pool_2(x)
            x = self.conv2(x)
            x = self.batch_norm_2(x)
            x = self.relu(x)
            x = self.max_pool_2(x)
            x = self.conv3(x)
            x = self.batch_norm_3(x)
            x = self.relu(x)
            x = self.max_pool_2(x)
            x = self.conv4(x)
            x = self.batch_norm_4(x)
            x = self.max_pool_2(x)
            x = self.fc_conv(x)
            #print("Shape of x is {}".format(x.shape))
            x = x.view(x.shape[0], -1)
            
            cnn_embed_seq.append(x)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        return cnn_embed_seq

In [109]:
batch_size = 4
epochs = 10
log_interval = 5
learning_rate = 1e-4

In [110]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu") 
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(train_data, **params)
model = Meso4()
model.cuda()
# for index, (X,y) in enumerate(train_loader):
#     y_pred = model(X)
#     print (y_pred.shape)
# Shape was torch.Size([4, 10, 2])



Meso4(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool_2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (max_pool_4): MaxPool2d(kernel_size=(4, 4), stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc_conv): Conv2d(16, 2, kernel_size=(16, 16), stride=(1, 1))
)

In [111]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [162]:
def find_median(numpy_array = []): #This is expected to take an array of array. So,
    #print("Input array is {}".format(numpy_array))
    output = []
    for array in numpy_array:
        counts = np.bincount(array)
        output.append(np.argmax(counts))
    return torch.from_numpy(np.asarray(output)).type(torch.LongTensor)

In [163]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train() #Put the model in training mode
    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device)

        N_count += X.size(0)

        optimizer.zero_grad()
        output = model(X)   # output has dim = (batch, number of classes)
        #print(output)
        #print(y.shape)
        loss_train = 0 
        for items in range(output.shape[1]):
            loss_train = loss_train + loss_function(output[:,items,:], y[:,items]).item()
        
        losses.append(loss_train)

        # to compute accuracy
        y_pred = torch.max(output, 2)[1]  # y_pred != output
        # We take the modal value of all predictions. So,
        y = find_median(y)
        y_pred = find_median(y_pred)
        print("Predicted is {}".format(y_pred))
        
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()

        # show information
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))

    return losses, scores

In [164]:
def validation(model, device, optimizer, test_loader):
    # set model as testing mode
    model.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device)

            output = model(X)
            print(output.shape)
            print(y.shape)
            loss = loss_function(output, y)
            test_loss += loss.item()                 # sum up batch loss
            y_pred = output.max(2, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability

            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
#     torch.save(cnn_encoder.state_dict(), os.path.join(save_model_path, 'cnn_encoder_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
#     torch.save(rnn_decoder.state_dict(), os.path.join(save_model_path, 'rnn_decoder_epoch{}.pth'.format(epoch + 1)))  # save motion_encoder
#     torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
#     print("Epoch {} model saved!".format(epoch + 1))

    return test_loss, test_score

In [165]:
for epoch in range(epochs):
    # train, test model
    train_losses, train_scores = train(log_interval, model, device, train_loader, optimizer, epoch)
    epoch_test_loss, epoch_test_score = validation(model, device, optimizer, valid_loader)

    # save results
    epoch_train_losses.append(train_losses)
    epoch_train_scores.append(train_scores)
    epoch_test_losses.append(epoch_test_loss)
    epoch_test_scores.append(epoch_test_score)


Predicted is tensor([1, 1, 0, 0])


NameError: name 'loss' is not defined

In [ ]:
# plot
fig = plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.plot(np.arange(1, epochs + 1), A[:, -1])  # train loss (on epoch end)
plt.plot(np.arange(1, epochs + 1), C)         #  test loss (on epoch end)
plt.title("model loss")
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train', 'test'], loc="upper left")
# 2nd figure
plt.subplot(122)
plt.plot(np.arange(1, epochs + 1), B[:, -1])  # train accuracy (on epoch end)
plt.plot(np.arange(1, epochs + 1), D)         #  test accuracy (on epoch end)
plt.title("training scores")
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train', 'test'], loc="upper left")
title = "./fig_UCF101_ResNetCRNN.png"
plt.savefig(title, dpi=600)
# plt.close(fig)
plt.show()

In [122]:
a = np.array([1,2,3,1,2,1,1,1,3,2,2,1])
counts = np.bincount(a)
print (np.argmax(counts))

1
